In [1]:
#Import necessary packages
import numpy as np
from scipy.integrate import quad
from scipy import stats
import warnings
import time
warnings.simplefilter('ignore')

#Install import_ipynb so we can read dependenies as jupyter notebooks
!pip install ipynb

#Import jupyer notebook depdendencies 
from ipynb.fs.full.FFT_option_valuation_LEWIS import BSM_call_value_FFT, H93_call_value_FFT
from ipynb.fs.full.Lewis_Integration_option_valuation import BSM_call_value, H93_call_value


In [2]:
#BSM analytical formula

def BSM_call_valueAnalytical(S0, K, T, r, d, sigma):
    ''' Valuation of European call option in BSM Model.
    Analytical Formula.
    Parameters
    ==========
    S0: float
    initial stock/index level
    K: float
    strike price
    T: float
    time-to-maturity (for t=0)
    r: float
    constant risk-free short rate
    d: float
    dividend yield
    sigma: float
    volatility factor in diffusion term
    Returns
    =======
    call_value: float
    European call option present value
    '''
    d1 = (np.log(S0 / K) + (r-d + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S0 / K) + (r-d - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    BS_C = (S0*np.exp(-d*T)* stats.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * stats.norm.cdf(d2, 0.0, 1.0))
    return BS_C


In [3]:
# BSM Parameters
S0 = 100.0
K = 100.0
T = 1.
r = 0.05
sigma = 0.2
d = 0

In [4]:

BSM_call_value(S0, K, T, r, d, sigma)

10.450583572184826

In [5]:
BSM_call_value_FFT(S0, K, T, r, d, sigma)

10.450813049892247

In [6]:
BSM_call_valueAnalytical(S0, K, T, r, d, sigma)

10.450583572185565

In [7]:
# Compare running time in one iteration
start_time = time.time()
BSM_call_valueAnalytical(S0, K, T, r, d, sigma)
print("BSM_Analytical method", (time.time() - start_time), "seconds")

start_time = time.time()
BSM_call_value(S0, K, T, r, d, sigma)
print("BSM_FT method", (time.time() - start_time), "seconds")

start_time = time.time()
BSM_call_value_FFT(S0, K, T, r, d, sigma)
print("BSM_FFT method", (time.time() - start_time), "seconds")

BSM_Analytical method 0.0005278587341308594 seconds
BSM_FT method 0.0027511119842529297 seconds
BSM_FFT method 0.001993894577026367 seconds


In [8]:
# Heston Parameters
S0 = 100.0
K = 100.0
T = 1.0
r = 0.05
kappa_v = 2
theta_v = 0.01
sigma_v = 0.1
rho = -0.5
v0 = 0.01
d = 0

In [9]:
H93_call_value(S0, K, T, r, d, kappa_v, theta_v, sigma_v, rho, v0)

6.874940187021409

In [10]:
H93_call_value_FFT(S0, K, T, r, d, kappa_v, theta_v, sigma_v, rho, v0)

6.87516966472441

In [11]:
import numpy.random as npr
np.random.seed(500)
def MC_Euler_Scheme_Heston_optionpricing(S0, K, T, r, d,
                                         v0, kappa, theta, sigma, rho,
                                         M, I, option = 'put', vt_disct = 'full-truncation'):
    ''' The Monte-Carlo estimation for a European option using the 
    standard Euler-Maruyama scheme in Heston Model
    Parameters
    ==========
    S0, K, T, r, d parameters as in typical European option.
    v0, kappa, theta, sigma, rho: Heston parameters for the volatility  
    M, I : (int) number of time steps and number of MC simulations
    option : (string) type of the option to be valued ('call', 'put')
    
    Returns
    =======
    C0 : float
        estimated present value of European put option'''
    
    ran_num = npr.standard_normal((2, M + 1, I)) 
    dt = T / M
    corr_mat = np.zeros((2, 2))
    corr_mat[0, :] = [1.0, rho]
    corr_mat[1, :] = [rho, 1.0]
    cho_mat = np.linalg.cholesky(corr_mat)
    cho_mat
    v = np.zeros_like(ran_num[0])
    vh = np.zeros_like(v)
    v[0] = v0
    vh[0] = v0
    S = np.zeros_like(ran_num[0])    
    S[0] = S0
    for t in range(1, M + 1):
    
        ran = np.dot(cho_mat, ran_num[:, t, :])
        if vt_disct == 'full-truncation':
        #this scheme is called full-truncation
            vh[t] = (vh[t - 1] + kappa * (theta - np.maximum(vh[t - 1], 0)) * dt
                  + sigma * np.sqrt(np.maximum(vh[t - 1], 0)) * np.sqrt(dt) * ran[1])
            v[t] = np.maximum(0, vh[t])
        
        #this scheme is called Reflection
        elif vt_disct == 'reflection':
            vh[t] = (vh[t - 1] + kappa * (theta - np.abs(vh[t - 1])) * dt
                  + sigma * np.sqrt(np.abs(vh[t - 1])) * np.sqrt(dt) * ran[1])
            v[t] = np.abs(vh[t])
        
        S[t] = S[t - 1] * np.exp((r -d - 0.5 * v[t]) * dt + np.sqrt(v[t]) * ran[0] * np.sqrt(dt))
    if option == 'call':
        hT = np.maximum(S[-1] - K, 0)
    elif option == 'put':
        hT = np.maximum(K - S[-1], 0)

    # calculation of MCS estimator for option price in Heston model
    option_price = np.exp(-(r-d) * T) * np.mean(hT)
    
    return option_price

In [12]:
# option parameter: the same as defined above for FT and FFT approach

# MC simulation parameters:
M = 252
I = 500000
MC_Euler_Scheme_Heston_optionpricing(S0, K, T, r, d,
                                     v0, kappa_v, theta_v, sigma_v, rho, 
                                     M, I, option = 'call', vt_disct = 'full-truncation')

5.190245637372806

In [13]:
# ITM option
S0 = 100.0
K = 85.0
T = 1.0
r = 0.05
kappa_v = 2
theta_v = 0.01
sigma_v = 0.1
rho = -0.5
v0 = 0.01
d = 0
MC_Euler_Scheme_Heston_optionpricing(S0, K, T, r, d,
                                     v0, kappa_v, theta_v, sigma_v, rho, 
                                     M, I, option = 'call', vt_disct = 'full-truncation')

16.896684595142407

In [14]:
H93_call_value(S0, K, T, r, d, kappa_v, theta_v, sigma_v, rho, v0)

19.27233648924421

In [15]:
H93_call_value_FFT(S0, K, T, r, d, kappa_v, theta_v, sigma_v, rho, v0)

19.27254918630409

In [16]:
# OTM option
S0 = 100.0
K = 115.0
T = 1.0
r = 0.05
kappa_v = 2
theta_v = 0.01
sigma_v = 0.1
rho = -0.5
v0 = 0.01
d = 0
MC_Euler_Scheme_Heston_optionpricing(S0, K, T, r, d,
                                     v0, kappa_v, theta_v, sigma_v, rho, 
                                     M, I, option = 'call', vt_disct = 'full-truncation')

0.47512869990502243

In [18]:
start_time = time.time()
MC_Euler_Scheme_Heston_optionpricing(S0, K, T, r, d,
                                     v0, kappa_v, theta_v, sigma_v, rho, 
                                     M, I, option = 'call')
print("Heston_MC", (time.time() - start_time), "seconds")
start_time = time.time()
H93_call_value(S0, K, T, r, d, kappa_v, theta_v, sigma_v, rho, v0)
print("Heston_Lewis", (time.time() - start_time), "seconds")
start_time = time.time()
H93_call_value_FFT(S0, K, T, r, d, kappa_v, theta_v, sigma_v, rho, v0)
print("Heston_FFT", (time.time() - start_time), "seconds")

Heston_MC 15.399959087371826 seconds
Heston_Lewis 0.007119655609130859 seconds
Heston_FFT 0.0024461746215820312 seconds


In [19]:
## Compare running time with 130 options in our benchmark

#Import CSV file with option data as a panda dataframe
import pandas as pd
options = pd.read_csv('option_data.csv')
options

,Maturity,Strike,Call,Date,T,r
0,2021-01-08,85.0,13.500,2020-12-28,0.030137,-0.000491
1,2021-01-08,87.5,11.000,2020-12-28,0.030137,-0.000491
2,2021-01-08,90.0,8.500,2020-12-28,0.030137,-0.000491
3,2021-01-08,92.5,6.000,2020-12-28,0.030137,-0.000491
4,2021-01-08,95.0,3.900,2020-12-28,0.030137,-0.000491
...,...,...,...,...,...,...
125,2021-12-17,120.0,4.125,2020-12-28,0.969863,0.000721
126,2021-12-17,130.0,2.425,2020-12-28,0.969863,0.000721
127,2021-12-17,140.0,1.375,2020-12-28,0.969863,0.000721
128,2021-12-17,150.0,0.825,2020-12-28,0.969863,0.000721


In [20]:
#Define Stock price (Ericsson B, 28.12.2020)               
S0 = 98.36   
#Dividend yield
d = 0.0086

In [21]:
start_time = time.time()
for row, option in options.iterrows():
    H93_call_value(S0, option['Strike'], option['T'],
                            option['r'], d, kappa_v, theta_v, sigma_v, rho, v0)
H93_call_value_time = (time.time() - start_time)
print(H93_call_value_time, "seconds")    
   

1.2030138969421387 seconds


In [22]:
start_time = time.time()
for row, option in options.iterrows():    
    H93_call_value_FFT(S0, option['Strike'], option['T'],
                            option['r'], d, kappa_v, theta_v, sigma_v, rho, v0)
H93_call_value_FFT_time = (time.time() - start_time)    
print(H93_call_value_FFT_time, "seconds")    

0.25907373428344727 seconds


In [ ]:
start_time = time.time()    
for row, option in options.iterrows():    
    MC_Euler_Scheme_Heston_optionpricing(S0, option['Strike'], option['T'], option['r'], d,
                                     v0, kappa_v, theta_v, sigma_v, rho, 
                                     M, I, option = 'call', vt_disct = 'full-truncation')
MC_Euler_Scheme_Heston_optionpricing_time = (time.time() - start_time)    
print(MC_Euler_Scheme_Heston_optionpricing_time, "seconds") 